In [75]:
from datetime import datetime
import pytz

In [129]:
import requests

# fill the parameters for the prediction
params = dict(
  pickup_datetime='2012-10-06 12:10:20',
  pickup_longitude=40.7614327,
  pickup_latitude=-73.9798156,
  dropoff_longitude=40.6413111,
  dropoff_latitude=-73.9797156,
  passenger_count=1
)

params = dict(
  pickup_datetime='2012-10-06 12:10:20',
  pickup_longitude=40.7614327,
  pickup_latitude=-73.9798156,
  dropoff_longitude=40.6331166,
  dropoff_latitude=-73.8874078,    
  passenger_count=2
)

# URL
taxifare_api_url = "http://127.0.0.1:8000/predict"

# retrieve the response
response = requests.get(
    taxifare_api_url,
    params=params
)

if response.status_code == 200:
    print("API call success")
else:
    print("API call error")

response.status_code, response.json().get("prediction", "no prediction"), response.json()


API call success


(200, 30.378084701440606, {'prediction': 30.378084701440606})

# Exploration creation dataframe à partir de dict

In [69]:
#from TaxiFareModel import trainer
import pandas as pd

In [80]:
params = dict(
  key = [''],
  pickup_datetime=['2012-10-06 12:10:20'],
  pickup_longitude=[40.7614327],
  pickup_latitude=[-73.9798156],
  dropoff_longitude=[40.6413111],
  dropoff_latitude=[-73.9797156],
  passenger_count=[1]
)

In [81]:
params

{'key': [''],
 'pickup_datetime': ['2012-10-06 12:10:20'],
 'pickup_longitude': [40.7614327],
 'pickup_latitude': [-73.9798156],
 'dropoff_longitude': [40.6413111],
 'dropoff_latitude': [-73.9797156],
 'passenger_count': [1]}

In [82]:
X_pred = pd.DataFrame.from_dict(params)

In [83]:
X_pred

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,,2012-10-06 12:10:20,40.761433,-73.979816,40.641311,-73.979716,1


In [84]:
type(X_pred)

pandas.core.frame.DataFrame

Example:


```
key                     object
pickup_datetime         object
pickup_longitude        float64
pickup_latitude         float64
dropoff_longitude       float64
dropoff_latitude        float64
passenger_count         int64

```

In [85]:
X_pred.dtypes

key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [102]:
# create a datetime object from the user provided datetime
pickup_datetime = datetime.strptime(X_pred['pickup_datetime'][0], "%Y-%m-%d %H:%M:%S")

# localize the user datetime with NYC timezone
eastern = pytz.timezone("US/Eastern")
localized_pickup_datetime = eastern.localize(pickup_datetime, is_dst=None)
#localized_pickup_datetime = pd.tz_localize(eastern)

# localize the datetime to UTC
utc_pickup_datetime = localized_pickup_datetime.astimezone(pytz.utc)
X_pred['key'] = utc_pickup_datetime.strftime("%Y-%m-%d %H:%M:%S UTC")
X_pred

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-10-06 16:10:20 UTC,2012-10-06 12:10:20,40.761433,-73.979816,40.641311,-73.979716,1


In [87]:
X_pred.dtypes

key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [118]:
X_pred.index=pd.DatetimeIndex(X_pred.index).tz_localize('UTC').tz_convert('US/Eastern')
X_pred.dtypes

key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [119]:
def generate_submission_for_API(X_pred):
    pipeline = joblib.load(PATH_TO_LOCAL_MODEL)
    if "best_estimator_" in dir(pipeline):
        y_pred = pipeline.best_estimator_.predict(X_pred)
    else:
        y_pred = pipeline.predict(X_pred)
    return y_pred

In [120]:
import joblib
PATH_TO_LOCAL_MODEL = '../model.joblib'

In [121]:
y_pred = generate_submission_for_API(X_pred)

TypeError: Cannot convert tz-naive timestamps, use tz_localize to localize